In [ ]:
import csv
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
csv_data = list(csv.reader(open('../Datasets/MindBigData-Imagenet/MindBigData_Imagenet_Insight_n00007846_112420_1_1479.csv', 'r')))

# data = np.zeros((len(csv_data), len(csv_data[0])-1))
# for i in range(len(data)):
#     data[i] = csv_data[i][1:]
data = np.array(csv_data)[:,1:].astype(float)

# plot 
plt.figure(figsize=(10, 7))
for i in range(len(data)):
    plt.plot(data[i], label='Image '+str(i))
plt.legend()
plt.title('ImageNet Insight')
plt.xlabel('Features')
plt.ylabel('Values')
plt.show()

In [ ]:
path = '../Datasets/MindBigData-Imagenet'
len(os.listdir(path))
data = []
for item in tqdm(os.listdir(path)):
    path_i = os.path.join(path, item)
    csv_data = list(csv.reader(open(path_i, 'r')))
    np_data = np.array(csv_data)[:,1:].astype(float)
    data.append(np_data)
    # close file
    csv_data = None
data = np.array(data)
data.shape